# State Objects with History

Some state objects use a history, rather than the fields direct in the state, to record changes in an experiment's 
data over time.

In [ ]:
import random
from pprint import pp

from autora.state import Delta
from autora.state_history_delta_alternative import AlternateDeltaHistory

AlternateDeltaHistory()

AlternateDeltaHistory(history=[...], variables=None, conditions=None, experiment_data=None, model=None)

We create an empty state and add initial data using Deltas. Each time this state is updated, the values in the fields `variables`, `conditions` etc. are replaced:

In [ ]:
import pandas as pd

s = AlternateDeltaHistory() + Delta(conditions=dict(x=range(5))) + Delta(conditions=dict(x=range(5, 10)))
s.conditions

,x
0,5
1,6
2,7
3,8
4,9


But the history keeps a record of the changes: the initial state (emtpy), and then the two Deltas:

In [ ]:
s.history

History([AlternateDeltaHistory(history=[...], variables=None, conditions=None, experiment_data=None, model=None),
         {'conditions': {'x': range(0, 5)}},
         {'conditions': {'x': range(5, 10)}}])

We can reconstruct the state at any point by slicing the history and using the `.reconstruct` method, here after the 
first Delta but before the second:

In [ ]:
s.history[:2].reconstruct().conditions

,x
0,0
1,1
2,2
3,3
4,4


By adding additional metadata to the Deltas, we can make it easier to find particular states. This might be useful in
 a complex AutoRA cycle where different steps need different versions of the same data.   

In [ ]:
def apply_transformation_to_input_state(function):
    """Decorator which applies a transformation to the input state"""
    

In [ ]:
from autora.state_history_delta import delta_to_state_transformed
from autora.state import inputs_from_state

t = AlternateDeltaHistory(conditions=pd.DataFrame({"x": []}))

# TODO: At this step we need a function which we can use to concatenate all of the 
# TODO: raw condition data but return them as part of the state. We don't have the 
# TODO: necessary helper functions on the History yet.
@delta_to_state_transformed(lambda s: s.where(meta="raw"))   
@inputs_from_state
def experimentalist(conditions):
    possible_conditions = set(range(10))
    print(conditions)
    already_seen_conditions = set(conditions["x"])  
    allowed_conditions = possible_conditions - already_seen_conditions
    conditions_out = min(allowed_conditions)
    return Delta(conditions=dict(x=[conditions_out]))

for i in range(6):
    t = experimentalist(t)
    print(t.history)




Empty DataFrame
Columns: [x]
Index: []
[AlternateDeltaHistory(history=[...], variables=None, conditions=Empty DataFrame
Columns: [x]
Index: [], experiment_data=None, model=None), {'conditions': {'x': [0]}}]
   x
0  0
[AlternateDeltaHistory(history=[...], variables=None, conditions=Empty DataFrame
Columns: [x]
Index: [], experiment_data=None, model=None), {'conditions': {'x': [0]}}, {'conditions': {'x': [1]}}]
   x
0  1
[AlternateDeltaHistory(history=[...], variables=None, conditions=Empty DataFrame
Columns: [x]
Index: [], experiment_data=None, model=None), {'conditions': {'x': [0]}}, {'conditions': {'x': [1]}}, {'conditions': {'x': [0]}}]
   x
0  0
[AlternateDeltaHistory(history=[...], variables=None, conditions=Empty DataFrame
Columns: [x]
Index: [], experiment_data=None, model=None), {'conditions': {'x': [0]}}, {'conditions': {'x': [1]}}, {'conditions': {'x': [0]}}, {'conditions': {'x': [1]}}]
   x
0  1
[AlternateDeltaHistory(history=[...], variables=None, conditions=Empty DataFrame
